# **DEPENDENCIES**

In [7]:
# Local import
import sys
sys.path.append('..')

# pip import on colab
# !pip install git+https://github.com/adityaprakash-work/DreamWalker.git
# from google.colab import drive
# drive.mount('/content/drive')

import dreamwalker as dw

In [ ]:
import torch

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# **DATASET**

In [ ]:
# Run this cell to load from an online source
dataset_url = None
dataset_dir = None
dw.utils.datasets.download(dataset_url, dataset_dir)

In [ ]:
train_dir = None
valid_dir = None
if valid_dir is None:
    dataset = dw.utils.datasets.ImageStream(train_dir)
    train_loader, valid_loader = dw.utils.datasets.get_loaders(
        dataset, return_valid=True, valid_size=0.2
    )

else:
    train_dataset = dw.utils.datasets.ImageStream(train_dir)
    valid_dataset = dw.utils.datasets.ImageStream(valid_dir)
    train_loader = dw.utils.datasets.get_loader(train_dataset)
    valid_loader = dw.utils.datasets.get_loader(valid_dataset)

# **TRAINING**

In [ ]:
load_path = None
save_path = None

if load_path is None:
    model = dw.models.vqvae.ResVQVAE()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    trainer = dw.models.vqvae.VQVAETrainerTrainer(
        model,
        optimizer,
        train_loader,
        valid_loader,
        device=DEVICE,
        load_checkpoint_path=None,
    )
else:
    trainer = dw.models.vqvae.VQVAETrainerTrainer(
        train_loader=train_loader,
        valid_loader=valid_loader,
        device=DEVICE, 
        load_checkpoint_path=load_path,
    )

In [ ]:
epochs = 100
monitors = [dw.utils.training.BestModelCheckpoint(save_path=save_path)]

trainer.train(epochs, monitors=monitors)